In [1]:
import sqlite3
import numpy as np

In [2]:
conn = sqlite3.connect('sentence_bank.sqlite3')
cursor = conn.cursor()
cursor2 = conn.cursor()
cursor3 = conn.cursor()

Prepare Splits

In [3]:
cursor.execute('SELECT id FROM sentences ORDER BY id;')
sentence_ids = []

for row in cursor:
    sentence_ids += [row[0]]

In [4]:
np.random.seed(1337)
train = list(np.random.choice(sentence_ids, size=int(len(sentence_ids)*0.7), replace=False))
valid = np.random.choice(list(set(sentence_ids) - set(train)), size=int(len(sentence_ids)*0.1), replace=False)
test = list(set(sentence_ids) - set(train) - set(valid))

In [5]:
print 'Splits:', len(train), len(valid), len(test)
print len(sentence_ids)

Splits: 2558 365 732
3655


Data file preparation

In [13]:
train_file = open('features_train.txt', 'w')
valid_file = open('features_valid.txt', 'w')
test_file  = open('features_test.txt', 'w')

for sentence_id, in cursor.execute('SELECT * FROM sentences;'):
    sentence = []
    cursor2.execute('''
    SELECT words.word_id, word, pos_tag, tag
    FROM words
    JOIN pos_tags ON words.sentence_id = pos_tags.sentence_id AND words.word_id = pos_tags.word_id                       
    WHERE words.sentence_id = ? ORDER BY words.word_id''', (sentence_id,))
    
    for row in cursor2:
        result = []
        word_id = row[0]
        row = list(row[1:])
        
        # POS tags
        row[1] = 'POS_' + row[1]
            
        result += [' '.join(row[:-1])]
        
        # other tags are retrieved here
        for tag, in cursor3.execute('''
        SELECT wordnet_tag FROM wordnet_tags
        WHERE sentence_id = ? AND word_id = ?
        ''', (sentence_id, word_id)):
            result += ['WORDNET_' + tag]
            
#         for tag, in cursor3.execute('''
#         SELECT DISTINCT mesh_type FROM mesh_tags
#         WHERE sentence_id = ? AND word_id = ?
#         ''', (sentence_id, word_id)):
#             result += ['MESH_TYPE_' + tag]
            
#         for tag, in cursor3.execute('''
#         SELECT DISTINCT mesh_keyword FROM mesh_tags
#         WHERE sentence_id = ? AND word_id = ?
#         ''', (sentence_id, word_id)):
#             result += ['MESH_' + tag]
        
        result += [row[-1]]
        sentence += [' '.join(result)]
    
    if sentence_id in train:
        _file = train_file
    if sentence_id in valid:
        _file = valid_file
    if sentence_id in test:
        _file = test_file
    
    _file.write('\n'.join(sentence).encode('ascii', 'ignore') + '\n\n')

In [14]:
train_file.close()
valid_file.close()
test_file.close()